In [18]:
import pandas as pd
from bokeh.plotting import show, figure, output_file
from bokeh.sampledata import periodic_table

# periodic_table is a sub module of bokeh.sampledata, 
# containing some other structures, including the 
# elements DataFrame

In [19]:
# set elements to the dataframe

elements = periodic_table.elements

In [20]:
# elements has many columns, including 'atomic number' and 'density'

for column in elements.columns:
    print column

atomic number
symbol
name
atomic mass
CPK
electronic configuration
electronegativity
atomic radius
ion radius
van der Waals radius
IE-1
EA
standard state
bonding type
melting point
boiling point
density
metal
year discovered
group
period


In [21]:
# use a filter to select just the 82 first elements

elements = elements[elements['atomic number'] <= 82]

In [22]:
# .isnull detects whether a row has no value for 'melting point'
# it returns True if that's the case
# The ~ operator inverts True and False values
# This creates a filter that *excludes* elements without melting points

elements = elements[~pd.isnull(elements['melting point'])]

In [23]:
# This is a list comprehension, which generates a list based on 
# a for loop. In this case, each list entry is based on atomic mass,
# which is stored as a string in the DataFrame
# Sometimes atomic mass has [] in it to express uncertainty
# x.strip() gets rid of them, and returns a "clean" string
# float() makes it numeric for plotting purposes

mass = [float(x.strip("[]")) for x in elements['atomic mass']]

In [24]:
# replace the original values (strings) in the 'atomic mass' column
# with numeric values

elements['atomic mass'] = mass

In [25]:
# define a set of colors that will be mapped to melting point values

palette = list(reversed([
            "#67001f", "#b2182b", "#d6604d", "#f4a582",
            "#fddbc7", "#f7f7f7", "#d1e5f0", "#92c5de",
            "#4393c3", "#2166ac", "#053061"
        ]))

In [26]:
# find the interval of possible melting-point values

melting_points = elements['melting point']
low  = min(melting_points)
high = max(melting_points)

In [27]:
# Another list comprehension. For each value (x) in melting_points,
# finds its proportion of the melting-point range, convert to scale
# of 0-10, then make it an integer.

melting_point_inds = [int(10*(x-low)/(high-low)) for x in melting_points]

# then use this integer as an index into the list palette

meltingpointcolors = [palette[i] for i in melting_point_inds]

In [28]:
print meltingpointcolors

['#053061', '#2166ac', '#d1e5f0', '#fddbc7', '#67001f', '#053061', '#053061', '#053061', '#053061', '#053061', '#4393c3', '#4393c3', '#d1e5f0', '#053061', '#053061', '#053061', '#053061', '#053061', '#4393c3', '#d1e5f0', '#f7f7f7', '#f7f7f7', '#f7f7f7', '#92c5de', '#d1e5f0', '#d1e5f0', '#d1e5f0', '#92c5de', '#2166ac', '#053061', '#92c5de', '#4393c3', '#2166ac', '#053061', '#053061', '#053061', '#4393c3', '#d1e5f0', '#f7f7f7', '#f4a582', '#f4a582', '#fddbc7', '#fddbc7', '#f7f7f7', '#d1e5f0', '#92c5de', '#2166ac', '#2166ac', '#2166ac', '#4393c3', '#2166ac', '#053061', '#053061', '#053061', '#4393c3', '#92c5de', '#4393c3', '#92c5de', '#92c5de', '#92c5de', '#92c5de', '#4393c3', '#d1e5f0', '#d1e5f0', '#d1e5f0', '#d1e5f0', '#d1e5f0', '#d1e5f0', '#4393c3', '#f7f7f7', '#fddbc7', '#d6604d', '#b2182b', '#b2182b', '#d6604d', '#f4a582', '#f7f7f7', '#92c5de', '#053061', '#2166ac', '#2166ac']


In [29]:
# set file to store graphic and set interactivity options 
# omitting "wheel_zoom" (not a fan) in tools

output_file("elements.html", title = "elements.py example")
TOOLS = "pan,box_zoom,reset,resize,save"


# create a figure object and set its title and background attributes

p = figure(tools = TOOLS, toolbar_location = "left", plot_width = 1200)
p.title = "Density vs Atomic Weight of Elements (colored by melting point)"
p.background_fill = "#cccccc"  # light gray color


# Use circles to plot density as a function of atomic weight

p.circle(x = elements["atomic mass"],
         y = elements["density"],
         size = 12,
         color = meltingpointcolors,  # using the list of colors defined above
         line_color = "black",        # color of just the outline
         fill_alpha = 0.8)

# ... and text to indicate chemical symbols, with a slight offset to
# vertical position to avoid overlapping the circles

p.text  (x = elements["atomic mass"],
         y = elements["density"] + 0.5,  # add a little height 
         text = elements["symbol"],
         text_color = "#333333",
         text_align = "center",
         text_font_size = "10pt")

# modify the axis_label attribute of each axis
# modify the grid_line_color attribute of the grid

p.xaxis.axis_label = "atomic weight (amu)"
p.yaxis.axis_label = "density (g/cm^3)"
p.grid.grid_line_color = "white"

show(p)

In [30]:
# Create a key of the palette colors, by drawing circles vertically
# need to set x and y coordinates for each circle

# y_pos is a list of integers from 10 to 20 (upper portion of graph)
# x_pos is a constant value, low to keep it on the left
# temps are temperatures corresponding to each color of the palette

scale = range(0,11)
y_pos = [10 + i for i in scale]
x_pos = 8
temps = [int((x) / 10.0 * (high-low) + low ) for x in scale]

# draw each circle at x_pos and y_pos, with color from palette

p.circle(x = x_pos,
         y = y_pos,
         size = 12,
         color = palette,
         line_color = "black",
         fill_alpha = 0.8)

# next to each circle, add text with the corresponding temperature
# set 3 to the left of the circle, and align to the right

p.text(  x = x_pos - 3,
         y = y_pos,
         text = temps,
         text_font_size = "12pt",
         text_color = palette,
         text_align = "right",
         text_baseline = "middle")

show(p)